# **Process for Wheat Yield Phenotype Files**

The process involves concatenating all downloaded phenotype files (those containing the keyword "GrnYld") into the following format (All.csv):

```csv
Trial name,Occ,Loc_no,Country,Loc_desc,Cycle,Cid,Sid,Gen_name,Trait_no,Trait name,Value,EMS,SE,Unit,GID,Plot
24ESWYT,1,14002,ANGOLA,HUMPATA,2003,61665,1,LOCAL CHECK,144,GRAIN_YIELD,2.375,0.018,0.681,t/ha,,
24ESWYT,1,14002,ANGOLA,HUMPATA,2003,8195,5,RAYON F 89,144,GRAIN_YIELD,4.029,0.018,0.681,t/ha,,
24ESWYT,1,14002,ANGOLA,HUMPATA,2003,160278,68,TARACHI F 2000,144,GRAIN_YIELD,4.134,0.018,0.681,t/ha,,
```

Additionally, concatenate all corresponding CID, SID, and GID values into the following format (CidSidGid.csv):

```csv
CID,SID,GID
61665,1,304660
60115,215,3820651
73574,1908,3829327
74933,481,3833626
```

Combine the rows containing the keywords “SOWING_DATE” and “HARVEST_FINISHING_DATE” from files in the phenotype files that include “EnvData” in their names into two separate files, named SOWING_DATE.csv and HARVEST_FINISHING_DATE.csv, respectively, as follows:

```csv
Trial name,Occ,Loc_no,Country,Loc_desc,Cycle,Trait No,Trait name,Value,Unit,Year,Month,Day
24 ESWYT,1,14002,ANGOLA,HUMPATA,2003,3,SOWING_DATE,Mar 1 2003,date,2003,3,1
24 ESWYT,6,11109,ZIMBABWE,CHISIPITE,2003,3,SOWING_DATE,May 8 2003,date,2003,5,8
24 ESWYT,7,22620,PAKISTAN,SAKRAND,2003,3,SOWING_DATE,Dec 6 2003,date,2003,12,6
```

```csv
Trial name,Occ,Loc_no,Country,Loc_desc,Cycle,Trait No,Trait name,Value,Unit,Year,Month,Day
24 ESWYT,1,14002,ANGOLA,HUMPATA,2003,9,HARVEST_FINISHING_DATE,Jul 23 2003,date,2003,7,23
24 ESWYT,6,11109,ZIMBABWE,CHISIPITE,2003,9,HARVEST_FINISHING_DATE,Oct 27 2003,date,2003,10,27
24 ESWYT,8,22607,PAKISTAN,NARC ISLAMABAD,2003,9,HARVEST_FINISHING_DATE,Apr 28 2004,date,2004,4,28
```

In [44]:
import pandas as pd
import numpy as np

# Read the original CSV file
df = pd.read_csv('source_data/All.csv')

# Select the required columns
selected_columns = ['Trial name', 'Occ', 'Loc_no', 'Cycle', 'Cid', 'Sid', 'Value']

selected_df = df[selected_columns]

# Save the selected columns to a new CSV file
selected_df.to_csv('output/AllSelected.csv', index=False)

In [ ]:
import pandas as pd

# Read the AllSelected.CSV file
all_selected = pd.read_csv('output/AllSelected.csv')

# Read the CidSidGid.csv file
cid_sid_gid = pd.read_csv('source_data/CidSidGid.csv')

# Merge the two dataframes by matching Cid and Sid, and extract GID
merged = pd.merge(
    all_selected, 
    cid_sid_gid, 
    how='left', 
    left_on=['Cid', 'Sid'], 
    right_on=['CID', 'SID']
)

# Remove redundant CID and SID columns, keeping only the Gid data
merged = merged.drop(columns=['CID', 'SID'])

# Save the new CSV file
merged.to_csv('output/AllWithGid.csv', index=False)

print("New CSV file has been generated: AllWithGid.csv")

In [46]:
import pandas as pd

all_with_gid = pd.read_csv('output/AllWithGid.csv')

# Replace empty strings and values with only spaces with NaN
all_with_gid = all_with_gid.applymap(lambda x: np.nan if isinstance(x, str) and x.strip() == "" else x)

# Remove rows containing NaN
all_with_gid = all_with_gid.dropna()

# Ensure the GID column is renamed to Gid and convert to the correct format
all_with_gid = all_with_gid.rename(columns={'GID': 'Gid'})

# Format Gid column values as 'GID' followed by the integer value of Gid
all_with_gid['Gid'] = 'GID' + all_with_gid['Gid'].astype(int).astype(str)

all_with_gid.to_csv('output/AllWithGidDropMissingValues.csv', index=False)

In [47]:
import pandas as pd

allpheno_df = pd.read_csv("output/AllWithGidDropMissingValues.csv")

# Read SOWING_DATE.csv file and handle duplicate rows
sowing_date_df = pd.read_csv("source_data/SOWING_DATE.csv", dtype={"Year": int, "Month": int, "Day": int})
sowing_date_df.drop_duplicates(subset=["Occ", "Loc_no", "Cycle"], keep="first", inplace=True)

# Convert SOWING_DATE.csv data to a dictionary with (Occ, Loc_no, Cycle) as key and (Year, Month, Day) as values
sowing_date_dict = sowing_date_df.set_index(["Occ", "Loc_no", "Cycle"])[["Year", "Month", "Day"]].to_dict(orient="index")

# Define a function to lookup (Year, Month, Day) data based on (Occ, Loc_no, Cycle)
def get_date(row):
    key = (row["Occ"], row["Loc_no"], row["Cycle"])
    date_data = sowing_date_dict.get(key)
    if date_data:
        return int(date_data["Year"]), int(date_data["Month"]), int(date_data["Day"])
    return None, None, None

# Apply the get_date function using the apply method to generate SowYear, SowMonth, and SowDay columns
allpheno_df[["SowYear", "SowMonth", "SowDay"]] = allpheno_df.apply(get_date, axis=1, result_type="expand")

allpheno_df.to_csv("output/AllWithGidDropMissingValuesSow.csv", index=False)

In [48]:
import pandas as pd

allpheno_df = pd.read_csv("output/AllWithGidDropMissingValuesSow.csv")

# Read HARVEST_FINISHING_DATE.csv file and handle duplicate rows
sowing_date_df = pd.read_csv("source_data/HARVEST_FINISHING_DATE.csv", dtype={"Year": int, "Month": int, "Day": int})
sowing_date_df.drop_duplicates(subset=["Occ", "Loc_no", "Cycle"], keep="first", inplace=True)

# Convert HARVEST_FINISHING_DATE.csv data to a dictionary with (Occ, Loc_no, Cycle) as key and (Year, Month, Day) as values
sowing_date_dict = sowing_date_df.set_index(["Occ", "Loc_no", "Cycle"])[["Year", "Month", "Day"]].to_dict(orient="index")

# Define a function to lookup (Year, Month, Day) data based on (Occ, Loc_no, Cycle)
def get_date(row):
    key = (row["Occ"], row["Loc_no"], row["Cycle"])
    date_data = sowing_date_dict.get(key)
    if date_data:
        return int(date_data["Year"]), int(date_data["Month"]), int(date_data["Day"])
    return None, None, None

# Apply the get_date function using the apply method to generate HarYear, HarMonth, and HarDay columns
allpheno_df[["HarYear", "HarMonth", "HarDay"]] = allpheno_df.apply(get_date, axis=1, result_type="expand")

allpheno_df = allpheno_df.dropna()

# Ensure SowYear, SowMonth, SowDay, HarYear, HarMonth, HarDay columns are integers
allpheno_df[["SowYear", "SowMonth", "SowDay", "HarYear", "HarMonth", "HarDay"]] = allpheno_df[["SowYear", "SowMonth", "SowDay", "HarYear", "HarMonth", "HarDay"]].astype(int)

# Create SowDate and HarDate columns using datetime
allpheno_df['SowDate'] = pd.to_datetime(allpheno_df[['SowYear', 'SowMonth', 'SowDay']].astype(str).agg('-'.join, axis=1))
allpheno_df['HarDate'] = pd.to_datetime(allpheno_df[['HarYear', 'HarMonth', 'HarDay']].astype(str).agg('-'.join, axis=1))

# Calculate the date difference in days and store it in the "Days" column
allpheno_df['Days'] = (allpheno_df['HarDate'] - allpheno_df['SowDate']).dt.days

# Optionally remove intermediate columns SowDate and HarDate
allpheno_df.drop(['SowDate', 'HarDate'], axis=1, inplace=True)

# 保存结果到另一个CSV文件
allpheno_df.to_csv("output/AllWithGidDropMissingValuesSowHar.csv", index=False)

In [49]:
import pandas as pd

# Function to read values from a txt file and store them as a set
def read_txt_file(file_path):
    with open(file_path, 'r') as txt_file:
        values = txt_file.read().split()
    return set(values)

# Function to filter rows in a CSV file based on Gid values from the txt file
def filter_csv_file(input_file, txt_file, output_file):
    values_set = read_txt_file(txt_file)

    df = pd.read_csv(input_file)

    filtered_df = df[df['Gid'].isin(values_set)]

    filtered_df.to_csv(output_file, index=False)

if __name__ == "__main__":
    input_csv_file = "output/AllWithGidDropMissingValuesSowHar.csv"
    txt_file = "../2_Geno/output/genotype_ID.txt" 
    output_csv_file = "output/AllWithGidDropMissingValuesSowHarFilteredGid.csv"

    filter_csv_file(input_csv_file, txt_file, output_csv_file)

In [ ]:
import pandas as pd

file_before2020 = 'output/AllWithGidDropMissingValuesSowHarFilteredGid.csv'
file_weather_data = '../3_Env/output/IWIN_Weather_AgERA5_20210211.csv'
output_file = 'output/AllWithGidDropMissingValuesSowHarFilteredGidLoc.csv'

data_before2020 = pd.read_csv(file_before2020)
data_weather = pd.read_csv(file_weather_data)

locations_before2020 = set(data_before2020['Loc_no'])
locations_weather_data = set(data_weather['location'])

cleaned_data = data_before2020[data_before2020['Loc_no'].isin(locations_weather_data)]

deleted_data = data_before2020[~data_before2020['Loc_no'].isin(locations_weather_data)]

if not deleted_data.empty:
    print(f"Deleted {len(deleted_data)} rows where 'Loc_no' is not in 'location'.")
    deleted_file_name = 'output/deleted_data.csv'
    deleted_data.to_csv(deleted_file_name, index=False)
    print(f"Deleted rows saved to file: {deleted_file_name}")
else:
    print("All 'Loc_no' values exist in 'location'. No rows were deleted.")

if not cleaned_data.empty:
    print(f"Cleaned {len(cleaned_data)} rows where 'Loc_no' exists in 'location'.")
    cleaned_data.to_csv(output_file, index=False)
    print(f"Cleaned data saved to file: {output_file}")
else:
    print("No rows to clean. All 'Loc_no' values exist in 'location'.")

In [51]:
"删除不合理的值:1 收获日大于2020的(不掌握2020后的天气数据);2.生长周期大于300天的;3.表型值大于20."
import pandas as pd

# 读取CSV文件
df = pd.read_csv('output/AllWithGidDropMissingValuesSowHarFilteredGidLoc.csv')

# Define conditions for filtering
condition = (df['Days'] > 270) | (df['HarYear'] >= 2020 ) | (df['Value'] == "-" )  | (df['Value'] >= 20 )

filtered_df = df[condition]

filtered_df.to_csv('output/Unormal.csv', index=False)

# Remove rows that meet the conditions
df.drop(filtered_df.index, inplace=True)

df.to_csv('output/AllWithGidDropMissingValuesSowHarFilteredGidLocUnormal.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('output/AllWithGidDropMissingValuesSowHarFilteredGidLocUnormal.csv')

# Identify duplicate rows, keeping the first occurrence and deleting the rest
df_duplicates = df[df.duplicated(keep='first')]
df.drop_duplicates(keep='first', inplace=True)

# Print the first duplicate row
if not df_duplicates.empty:
    print("The first duplicate row:")
    print(df_duplicates.head())

# Save the removed duplicate rows to a new CSV file
df_duplicates.to_csv('output/Duplicate.csv', index=False)
print("The removed duplicate rows have been saved to Duplicate.csv.")

# Save the updated data back to the original file
df.to_csv('output/AllWithGidDropMissingValuesSowHarFilteredGidLocUnormalNoDuplicated.csv', index=False)
print("The updated data has been saved to AllWithGidDropMissingValuesSowHarFilteredGidLocUnormalNoDuplicated.csv.")

In [53]:
"提取去除重复后的环境型"
import pandas as pd

df = pd.read_csv('output/AllWithGidDropMissingValuesSowHarFilteredGidLocUnormalNoDuplicated.csv')

# Combine three columns into a new column
df['Occ_Loc_no_Cycle'] = df['Occ'].astype(str) + '_' + df['Loc_no'].astype(str) + '_' + df['Cycle'].astype(str)

df = df.drop_duplicates(subset='Occ_Loc_no_Cycle', keep='first')

df.to_csv('output/UniqueOccLocCycle.csv', columns=['Occ_Loc_no_Cycle'], index=False)

In [54]:
"提取去除重复后的基因型"
import pandas as pd

df = pd.read_csv('output/AllWithGidDropMissingValuesSowHarFilteredGidLocUnormalNoDuplicated.csv')

# Extract the Gid column and keep the first occurrence of each unique value
unique_gid_series = df['Gid'].drop_duplicates(keep='first')

new_df = pd.DataFrame({'Gid': unique_gid_series})

new_df.to_csv('output/UniqueGid.csv', index=False)